In [1]:
import numpy as np
import cdsapi
from pathlib import Path
import pandas as pd

In [2]:
path_thermal_data = Path("../data/input_data/Thermal_characteristics_beforeEE.csv")
residential_data = pd.read_csv(path_thermal_data, index_col=0)
residential_data

,Dwelling forms,Heating systems,Average thermal capacity kJ/K,Average thermal losses kW/K,Number of dwellings,Average annual heat demand kWh,Average floor area m2,Thermal capacity level,Design temperature degreeC,Region,LSOA_code,Local Authority
Index,,,,,,,,,,,,
0,flat,oil boiler,7760.997724,0.303952,12.0,12937.408384,77.609977,low,-2.0,London,E01000001,City of London
1,semi-detached,gas boiler,9761.746787,0.397347,2.0,16912.665792,97.617468,low,-2.0,London,E01000001,City of London
2,terraced,gas boiler,17588.048538,0.695063,1.0,29584.664638,175.880485,low,-2.0,London,E01000001,City of London
3,terraced,resistance heating,6526.000000,0.094418,10.0,4018.826439,65.260000,low,-2.0,London,E01000001,City of London
4,flat,gas boiler,5785.363636,0.152747,95.0,6501.515152,57.853636,low,-2.0,London,E01000001,City of London
...,...,...,...,...,...,...,...,...,...,...,...,...
1036984,terraced,gas boiler,45775.554843,0.373869,107.0,18474.199623,101.723455,high,-3.0,Wales,W01001958,Swansea
1036985,terraced,resistance heating,39931.500000,0.193215,2.0,9547.432550,88.736667,high,-3.0,Wales,W01001958,Swansea
1036986,flat,gas boiler,23860.102500,0.160746,370.0,7943.055556,53.022450,high,-3.0,Wales,W01001958,Swansea


In [3]:
## Import the required library
# from geopy.geocoders import Nominatim
# coordinates_LA = {}

# for LA in residential_data["Local Authority"].unique():
#     # Initialize Nominatim API
#     geolocator = Nominatim(user_agent="MyApp")
#     location = geolocator.geocode(LA)
#     coordinates_LA[LA] = (location.latitude, location.longitude)
#     # print(f"{LA} -- the latitude is: {location.latitude} and the longitude is: {location.longitude}.")

# coordinates_LA_df = pd.DataFrame(coordinates_LA).T
# coordinates_LA_df.columns = ["Latitude", "Longitude"]
# coordinates_LA_df.to_csv(r"../data/intermediary/coordinates_LA.csv")
# coordinates_LA_df.head()

In [4]:
## Load coordinates data

coordinates_LA_df = pd.read_csv(r"../data/intermediary/coordinates_LA.csv", index_col = 0)
coordinates_LA_df

,Latitude,Longitude
City of London,51.515618,-0.091998
Barking and Dagenham,51.554091,0.150489
Barnet,51.653090,-0.200226
Bexley,39.969238,-82.936864
Brent,51.563996,-0.275906
...,...,...
Blaenau Gwent,51.753382,-3.184856
Torfaen,51.701480,-3.052050
Monmouthshire,51.750106,-2.833333
Newport,41.489983,-71.313771


['Havering',
 'Hillingdon',
 'Hounslow',
 'Islington',
 'Kensington and Chelsea',
 'Kingston upon Thames',
 'Lambeth',
 'Lewisham',
 'Merton',
 'Newham',
 'Redbridge',
 'Richmond upon Thames',
 'Southwark',
 'Sutton',
 'Tower Hamlets',
 'Waltham Forest',
 'Wandsworth',
 'Westminster',
 'Bolton',
 'Bury',
 'Manchester',
 'Oldham',
 'Rochdale',
 'Salford',
 'Stockport',
 'Tameside',
 'Trafford',
 'Wigan',
 'Knowsley',
 'Liverpool',
 'St. Helens',
 'Sefton',
 'Wirral',
 'Barnsley',
 'Doncaster',
 'Rotherham',
 'Sheffield',
 'Gateshead',
 'Newcastle upon Tyne',
 'North Tyneside',
 'South Tyneside',
 'Sunderland',
 'Birmingham',
 'Coventry',
 'Dudley',
 'Sandwell',
 'Solihull',
 'Walsall',
 'Wolverhampton',
 'Bradford',
 'Calderdale',
 'Kirklees',
 'Leeds',
 'Wakefield',
 'Hartlepool',
 'Middlesbrough',
 'Redcar and Cleveland',
 'Stockton-on-Tees',
 'Darlington',
 'Halton',
 'Warrington',
 'Blackburn with Darwen',
 'Blackpool',
 'Kingston upon Hull',
 'East Riding of Yorkshire',
 'North Eas

In [12]:
# http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000 was used to get the coordinates of the location, the input coordinate data is a box [N, W, E,S] 
# where N the latitude of the top of the box, W the longitude of the left of the box, E the longitude of the left of the box and S the bottom of the box.
# https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form was used to generate the code
# select netcdf format
c = cdsapi.Client()

list_las= list(coordinates_LA_df.index)
for LA in list_las:
    box_coordinates = coordinates_LA_df.loc[LA].to_list() + [x-0.01 if x>0 else x+0.01 for x in coordinates_LA_df.loc[LA].to_list()]
    box_coordinates = [round(x, 2) for x in box_coordinates]
    print(LA, box_coordinates)
    path_project:str = str(Path().absolute()).split("code")[0]

    path_netcdf_files = Path(path_project)/"data"/f"{LA}_netcdf_data"
    path_netcdf_files.mkdir(parents=True, exist_ok=True)


    list_years = [2022]# np.arange(2010, 2022)

    list_variables = ['surface_net_solar_radiation', '2m_temperature']#,  '10m_u_component_of_wind', '10m_v_component_of_wind' #


    for year in list_years:
        for variable in list_variables:
            filename = f'{LA}_{variable}_{year}.nc'
            temp_full_path = path_netcdf_files/filename
            temp_result = c.retrieve(
                'reanalysis-era5-land',
                {
                    'format': 'netcdf',
                    'variable': f'{variable}',
                    'year': f'{year}',
                    'month': [
                        '01','02', '03',
                        '04', '05', '06',
                        '07', '08', '09',
                        '10', '11', '12',
                    ],
                    'day': [
                        '01', '02', '03',
                        '04', '05', '06',
                        '07', '08', '09',
                        '10', '11', '12',
                        '13', '14', '15',
                        '16', '17', '18',
                        '19', '20', '21',
                        '22', '23', '24',
                        '25', '26', '27',
                        '28', '29', '30',
                        '31',
                    ],
                    'time': [
                        '00:00', '01:00', '02:00',
                        '03:00', '04:00', '05:00',
                        '06:00', '07:00', '08:00',
                        '09:00', '10:00', '11:00',
                        '12:00', '13:00', '14:00',
                        '15:00', '16:00', '17:00',
                        '18:00', '19:00', '20:00',
                        '21:00', '22:00', '23:00',
                    ],
                    'area': box_coordinates,
                },
                f'{temp_full_path}')

Havering [51.54, -0.14, 51.53, -0.13]


2022-10-26 16:42:17,625 INFO Welcome to the CDS
2022-10-26 16:42:17,626 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2022-10-26 16:42:17,829 INFO Request is completed
2022-10-26 16:42:17,830 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data1/adaptor.mars.internal-1666795480.9092739-32425-15-fcbd57e6-43bb-4ea3-9bab-3d0724fa71e5.nc to c:\Users\sceac10\OneDrive - Cardiff University\General\data\Havering_netcdf_data\Havering_surface_net_solar_radiation_2022.nc (31K)
2022-10-26 16:42:18,137 INFO Download rate 101K/s
2022-10-26 16:42:18,185 INFO Welcome to the CDS
2022-10-26 16:42:18,186 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2022-10-26 16:42:18,239 INFO Request is queued


Hillingdon [51.54, -0.45, 51.53, -0.44]


2022-10-26 16:42:19,291 INFO Request is running
2022-10-26 16:56:37,619 INFO Request is completed
2022-10-26 16:56:37,620 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data5/adaptor.mars.internal-1666799681.2533505-9562-8-040b6422-23f4-4a20-9ae1-77a91dced4bb.nc to c:\Users\sceac10\OneDrive - Cardiff University\General\data\Hillingdon_netcdf_data\Hillingdon_surface_net_solar_radiation_2022.nc (30.9K)
2022-10-26 16:56:37,855 INFO Download rate 131.7K/s
2022-10-26 16:56:37,969 INFO Welcome to the CDS
2022-10-26 16:56:37,969 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2022-10-26 16:56:38,038 INFO Request is queued


Hounslow [51.47, -0.36, 51.46, -0.35]


...

2022-10-30 06:35:27,066 INFO Request is running
2022-10-30 06:47:45,065 INFO Request is completed
2022-10-30 06:47:45,066 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data8/adaptor.mars.internal-1667112436.036181-19994-10-0bfccea7-719a-4c2f-8236-6b86c689415a.nc to c:\Users\sceac10\OneDrive - Cardiff University\General\data\Cardiff_netcdf_data\Cardiff_surface_net_solar_radiation_2022.nc (30.9K)
2022-10-30 06:47:45,322 INFO Download rate 121.4K/s
2022-10-30 06:47:45,421 INFO Welcome to the CDS
2022-10-30 06:47:45,422 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2022-10-30 06:47:45,496 INFO Request is queued


City of London [51.52, -0.09, 51.51, -0.08]


2022-10-30 06:47:46,570 INFO Request is running
2022-10-30 07:02:04,778 INFO Request is completed
2022-10-30 07:02:04,778 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data0/adaptor.mars.internal-1667113198.9777832-6836-4-86f93281-a50e-4926-abf5-e41b3bfbee9d.nc to c:\Users\sceac10\OneDrive - Cardiff University\General\data\City of London_netcdf_data\City of London_surface_net_solar_radiation_2022.nc (30.9K)
2022-10-30 07:02:05,008 INFO Download rate 135.2K/s
2022-10-30 07:02:05,105 INFO Welcome to the CDS
2022-10-30 07:02:05,106 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2022-10-30 07:02:05,182 INFO Request is queued


Barking and Dagenham [51.55, 0.15, 51.54, 0.14]


2022-10-30 07:02:06,238 INFO Request is running
2022-10-30 07:14:24,390 INFO Request is completed
2022-10-30 07:14:24,391 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data9/adaptor.mars.internal-1667114045.7847779-1103-17-3c46ae77-1723-4404-8975-9b28e03dbc26.nc to c:\Users\sceac10\OneDrive - Cardiff University\General\data\Barking and Dagenham_netcdf_data\Barking and Dagenham_surface_net_solar_radiation_2022.nc (35M)
2022-10-30 07:14:30,558 INFO Download rate 5.7M/s   
